# Workflow

- In this Notebook I'm working on:
    - IaC (create IAM, Redshift cluster)
    - Executing ETL Pipeline
    - Testing Final data by quering the loaded tables
    - Cleanup and deleteing used resources

## Setup

In [10]:
import pandas as pd
import boto3
import json

In [11]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("CLUSTER","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("CLUSTER","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("CLUSTER","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("CLUSTER","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("CLUSTER","DWH_DB")
DWH_DB_USER            = config.get("CLUSTER","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DWH_DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("CLUSTER", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dwhCluster
4,DWH_DB,dwh
5,DWH_DB_USER,dwhuser
6,DWH_DB_PASSWORD,Passw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dwhRole


## IaC

## Create clients for S3, IAM, Redshift

In [36]:
import boto3

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

## Create IAM ROLE
- Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [ ]:
#1.1 Create the IAM role
try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(Path='/',
                             RoleName=DWH_IAM_ROLE_NAME,
                             Description="Allows Redshift clusters to call AWS services on your behalf.",
                             AssumeRolePolicyDocument=json.dumps(
                            {'Statement': [{'Action': 'sts:AssumeRole',
                               'Effect': 'Allow',
                               'Principal': {'Service': 'redshift.amazonaws.com'}}],
                             'Version': '2012-10-17'}))
    

except Exception as e:
    print(e)
    
#1.2 Attach Policy    
print('1.2 Attaching Policy')
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

#1.3 Get and print the IAM role ARN
print('1.3 Get the IAM role ARN')
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

## Create Redshift Cluster

- Create a RedShift Cluster

In [6]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn] 
         
    )
except Exception as e:
    print(e)

In [7]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.cicguwddygfd.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-014c33f6165d3c292
7,NumberOfNodes,4


## Execute ETL on cluster

In [ ]:
# Execute ETL on cluster
%run create_tables.py
%run etl.py

## Testing

In [22]:
# Connect to database
conn_str="host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values())
conn = psycopg2.connect(conn_str)
cur = conn.cursor()

### Querying Statging Tables

In [25]:
%%time
cur.execute("SELECT COUNT(*) FROM staging_events;")
print(cur.fetchall()[0])

(8056,)
CPU times: user 1.12 ms, sys: 39 µs, total: 1.16 ms
Wall time: 65.6 ms


In [26]:
%%time
cur.execute("SELECT COUNT(*) FROM staging_songs;")
print(cur.fetchall()[0])


(14896,)
CPU times: user 1.24 ms, sys: 27 µs, total: 1.27 ms
Wall time: 83.6 ms


### Querying Analytics Tables

In [27]:
%%time
cur.execute("SELECT COUNT(*) FROM users;")
print(cur.fetchall()[0])


(104,)
CPU times: user 1.53 ms, sys: 0 ns, total: 1.53 ms
Wall time: 215 ms


In [28]:
%%time
cur.execute("SELECT COUNT(*) FROM artists;")
print(cur.fetchall()[0])


(10025,)
CPU times: user 1.4 ms, sys: 309 µs, total: 1.71 ms
Wall time: 86.1 ms


In [29]:
%%time
cur.execute("SELECT COUNT(*) FROM time;")
print(cur.fetchall()[0])


(6813,)
CPU times: user 993 µs, sys: 219 µs, total: 1.21 ms
Wall time: 84.8 ms


In [30]:
%%time
cur.execute("SELECT COUNT(*) FROM songplays;")
print(cur.fetchall()[0])


(9957,)
CPU times: user 1.64 ms, sys: 0 ns, total: 1.64 ms
Wall time: 82.6 ms


In [34]:

cur.execute("""
    SELECT  sp.songplay_id,
        u.user_id,
        u.last_name,
        u.first_name,
        sp.start_time,
        a.name,
        s.title
FROM songplays AS sp
        JOIN users   AS u ON (u.user_id = sp.user_id)
        JOIN songs   AS s ON (s.song_id = sp.song_id)
        JOIN artists AS a ON (a.artist_id = sp.artist_id)
        JOIN time    AS t ON (t.start_time = sp.start_time)
ORDER BY (u.last_name)
LIMIT 100;
""")
pd.DataFrame(cur.fetchall())


,0,1,2,3,4,5,6
0,895,66,Arellano,Kevin,2018-11-23 00:11:00,Soziedad Alkoholika,Rumore
1,10210,66,Arellano,Kevin,2018-11-23 05:56:58,The Cinematic Orchestra/London Metropolitan Or...,Dawn
2,8379,66,Arellano,Kevin,2018-11-13 00:40:37,Fu,To Wszystko Co Mam
3,6174,66,Arellano,Kevin,2018-11-12 11:41:09,Devin Townsend,Hidden Track
4,3834,66,Arellano,Kevin,2018-11-06 07:36:46,Basshunter feat. DJ Mental Theos Bazzheadz,Every Morning
5,264,66,Arellano,Kevin,2018-11-12 19:10:42,Evanescence,Missing (Live in Europe)
6,3487,66,Arellano,Kevin,2018-11-07 12:06:23,Matt Nathanson,Suspended
7,578,66,Arellano,Kevin,2018-11-23 00:11:00,Soziedad Alkoholika,El Maximo Daño
8,3834,66,Arellano,Kevin,2018-11-06 07:36:46,Basshunter,Every Morning
9,3889,66,Arellano,Kevin,2018-11-06 07:36:46,Basshunter feat. DJ Mental Theos Bazzheadz,Why


## Cleanup

In [37]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

{'Cluster': {'ClusterIdentifier': 'dwhcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'MasterUsername': 'dwhuser',
  'DBName': 'dwh',
  'Endpoint': {'Address': 'dwhcluster.cicguwddygfd.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2022, 8, 7, 20, 58, 6, 921000, tzinfo=tzlocal()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0932b1027f8918f27',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-014c33f6165d3c292',
  'AvailabilityZone': 'us-west-2d',
  'PreferredMaintenanceWindow': 'wed:08:00-wed:08:30',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRoutin

In [38]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!

{'ResponseMetadata': {'RequestId': 'eeba8402-aefd-4fbe-8abb-91087c7a8fbc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'eeba8402-aefd-4fbe-8abb-91087c7a8fbc',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Sun, 07 Aug 2022 21:40:15 GMT'},
  'RetryAttempts': 0}}